# Sample Translation

Here, we first clone github repository to use IndicTRANS model, preprocess the story according to the model need and try to translate one story of मुंशी प्रेमचन्द into english.

## General setup for the model

Just run this snippet only once and then restart the runtime.

In [1]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 503, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 503 (delta 163), reused 132 (delta 118), pack-reused 297
Receiving objects: 100% (503/503), 1.49 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (290/290), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 11.42 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 

In [2]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install --editable ./

%cd ..

     |████████████████████████████████| 895 kB 5.3 MB/s 
     |████████████████████████████████| 90 kB 9.0 MB/s 
     |████████████████████████████████| 124 kB 67.2 MB/s 
     |████████████████████████████████| 40 kB 4.6 MB/s 
     |████████████████████████████████| 2.8 MB 48.0 MB/s 
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-py3-none-any.whl size=49117 sha256=66ba8720bdcb986d7c82cec3e839c9a014021c8de7a83aef2ca48f4b0d87bf04
  Stored in directory: /root/.cache/pip/wheels/a7/31/94/fef279382208e85a65c1a7f5c4d0020115477b0af74f296b57
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3354 sha256=94e6c70104d61c03bc68bb3a763375d81e1702562c49176a879846acf64a421e
  Stored in directory: /root/.cache/pip/wheels/c5/4f/33/54741ffe08e38ececb1d28068a153729b4fe820bafa0a0691f
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6163 sha256=527b9b83328fd6c81c26bb20fee37aced19d6805eef1976c3f39a004fcb72462
  Stored in directory: /r

**Restart your runtime and start execution from here on...**

In [1]:
# extra code for colab file (don't run if not working on one)
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [2]:
!wget https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
!unzip indic-en.zip

%cd indicTrans

--2021-09-25 03:45:46--  https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.128, 142.250.136.128, 142.250.148.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551079075 (4.2G) [application/zip]
Saving to: ‘indic-en.zip’

indic-en.zip        100%[===================>]   4.24G  42.1MB/s    in 78s     

2021-09-25 03:47:04 (55.3 MB/s) - ‘indic-en.zip’ saved [4551079075/4551079075]

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin/dict.SRC.txt  
   creating: indic-en/model/
  

In [3]:
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../indic-en')

Initializing vocab and bpe
Initializing model for translation


2021-09-25 03:49:11 | INFO | fairseq.tasks.translation | [SRC] dictionary: 35864 types
2021-09-25 03:49:11 | INFO | fairseq.tasks.translation | [TGT] dictionary: 32088 types


**The model has been setup**

## Preprocessing of the story according the model need.

This model can works on sentence tokenized version of story. So we will be tokenising the story sentence-wise.

So we are working on the story **कफ़न**.

In [4]:
story = '''झोपड़े के द्वार पर बाप और बेटा दोनों एक बुझे हुए अलाव के और अन्दर बेटे कि जवान बीवी बुधिया प्रसव-वेदना से पछाड़ खा रही थी। रह-रहकर उसके मुँह से ऐसी दिल हिला देने वाली आवाज़ निकलती थी, कि दोनों कलेजा थाम लेते थे। जाड़े की रात थी, प्रकृति सन्नाटे में डूबी हुई, सारा गाँव अन्धकार में लय हो गया था।

घीसू ने कहा – मालूम होता है, बचेगी नहीं। सारा दिन दौड़ते ही गया, ज़रा देख तो आ।

माधव चिढ़कर बोला – मरना ही है तो जल्दी मर क्यों नही जाती ? देखकर क्या करूं?

‘तू बड़ा बेदर्द है बे ! साल-भर जिसके साथ सुख-चैन से रहा, उसी के साथ इतनी बेवफाई!’ ‘तो मुझसे तो उसका तड़पना और हाथ-पाँव पटकना नहीं देखा जाता।’

चमारों का कुनबा था और सारे गाँव में बदनाम। घीसू एक दिन काम करता तो तीन दिन आराम करता। माधव इतना कामचोर था कि आधे घंटे काम करता तो घंटे भर चिलम पीता। इसीलिये उन्हें कहीँ मज़दूरी नहीं मिलती थी। घर में मुट्ठी भर अनाज भी मौजूद हो, तो उनके लिए काम करने कि कसम थी। जब दो-चार फाके हो जाते तो घीसू पेड़ पर चढ़कर लकड़ियां तोड़ लाता और माधव बाज़ार में बेच आता। जब तक वह पैसे रहते, दोनों इधर उधर मारे-मारे फिरते। गाँव में काम कि कमी ना थी। किसानों का गाँव था, मेहनती आदमी के लिए पचास काम थे। मगर इन दोनों को उस वक़्त बुलाते, जब दो आदमियों से एक का काम पाकर भी संतोष कर लेने के सिवा और कोई चारा ना होता। अगर दोनों साधू होते, तो उन्हें सुन्तोष और धैर्य के लिए, संयम और नियम की बिल्कुल ज़रूरत न होती। यह तो इनकी प्रकृति थी। विचित्र जीवन था इनका! घर में मिट्टी के दो-चार बर्तन के सिवा और कोई सम्पत्ति नही थी। फटे चीथ्डों से अपनी नग्नता को ढांके हुए जीये जाते थे। संसार की चिंताओं से मुक्त! कर्ज़ से लदे हुए। गालियाँ भी खाते, मार भी खाते, मगर कोई गम नहीं। दीं इतने की वसूली की बिल्कुल आशा ना रहने पर भी लोग इन्हें कुछ न कुछ कर्ज़ दे देते थे। मटर, आलू कि फसल में दूसरों के खेतों से मटर या आलू उखाड़ लाते और भून-भूनकर खा लेते या दुस-पांच ईखें उखाड़ लाते और रात को चूसते। घीसू ने इसी आकाश-वृति से साठ साल कि उम्र काट दी और माधव भी सपूत बेटे कि तरह बाप ही के पद चिन्हों पर चल रहा था, बल्कि उसका नाम और भी उजागर कर रहा था। इस वक़्त भी दोनो अलाव के सामने बैठकर आलू भून रहे थे, जो कि किसी खेत से खोद लाए थे। घीसू की स्त्री का तो बहुत दिन हुए देहांत हो गया था। माधव का ब्याह पिछले साल हुआ था। जबसे यह औरत आयी थी, उसने इस खानदान में व्यवस्था की नींव डाली थी और इन दोनो बे-गैरतों का दोजख भारती रहती थी। जब से वोह आयी, यह दोनो और भी आराम तलब हो गए थे। बल्कि कुछ अकडने भी लगे थे। कोई कार्य करने को बुलाता, तो निर्बयाज भाव से दुगनी मजदूरी माँगते। वही औरत आज प्रसव-वेदना से मर रही थी, और यह दोनों शायद इसी इंतज़ार में थे कि वोह मर जाये, तो आराम से सोयें।

घीसू ने आलू छीलते हुए कहा- जाकर देख तो, क्या दशा है उसकी? चुड़ैल का फिसाद होगा, और क्या! यहाँ तो ओझा भी एक रुपया माँगता है!

माधव तो भय था कि वोह कोठरी में गया, तो घीसू आलू का एक बड़ा भाग साफ कर देगा। बोला- मुझे वहाँ जाते डर लगता है।

‘डर किस बात का है, मैं तो यहाँ हूँ ही।’ ‘तो तुम्ही जाकर देखो ना।’

‘मेरी औरत जब मरी थी, तो मैं तीन दिन तक उसके पास से हिला तक नही; और फिर मुझसे लजायेगा कि नहीं? जिसका कभी मुँह नही देखा; आज उसका उधडा हुआ बदन देखूं। उसे तन कि सुध भी तो ना होगी। मुझे देख लेगी तो खुलकर हाथ-पाँव भी ना पटक सकेगी!’

‘मैं सोचता हूँ, कोई बाल बच्चा हुआ, तो क्या होगा? सोंठ, गुड, तेल, कुछ भी तो नही है घर में!’

‘सब कुछ आ जाएगा। भगवान् दे तो! जो लोग अभी एक पैसा नहीं दे रहे हैं, वो ही कल बुलाकर रुपये देंगे। मेरे नौ लड़के हुए, घर में कभी कुछ ना था, भगवान् ने किसी ना किसी तरह बेडा पार ही लगाया।’

जिस समाज में रात-दिन म्हणत करने वालों कि हालात उनकी हालात से कुछ अच्छी ना थी, और किसानों के मुकाबले में वो लोग, जो किसानों कि दुर्बलताओं से लाभ उठाना जानते थे, कहीँ ज़्यादा सम्पन्न थे, वहाँ इस तरह की मनोवृति का पैदा हो जान कोई अचरज की बात नहीं थी। हम तो कहेंगे, घीसू किसानों से कहीँ ज़्यादा विचारवान था और किसानों के विचार-शुन्य समूह में शामिल होने के बदले बैठक बाजों की कुत्सित मंडळी में जा मिलता था। हाँ, उसमें यह शक्ति ना थी कि बैठक बाजों के नियम और निति का पालन कर्ता। इसलिये जहाँ उसकी मंडळी के और लोग गाव के सरगना और मुखिया बने हुए थे, उस पर सारा गाव ऊँगली उठाता था। फिर भी उसे यह तस्कीन तो थी ही, कि अगर वोह फटेहाल हैं तो उसे किसानों की-सी जी-तोड़ म्हणत तो नही करनी पड़ती, और उसकी सरलता और निरीहता से दुसरे लोग बेजा फायदा तो नही उठाते। दोनो आलू निकल-निकलकर जलते-जलते खाने लगे। कल से कुछ नही खाया था। इतना सब्र ना था कि उन्हें ठण्डा हो जाने दे। कई बार दोनों की ज़बान जल गयी। चिल जाने पर आलू का बहरी हिस्सा बहुत ज़्यादा गरम ना मालूम होता, लेकिन दोनों दांतों के तले पड़ते ही अन्दर का हिस्सा ज़बान, तलक और तालू जला देता था, और उस अंगारे को मुँह में रखेने से ज़्यादा खैरियत तो इसी में थी कि वो अन्दर पहुंच जाये। वहाँ उसे ठण्डा करने के लिए काफी समान था। इसलिये दोनों जल्द-जल्द निगल जाते । हालांकि इस कोशिश में उन्ही आंखों से आँसू निकल आते ।

घीसू को उस वक़्त ठाकुर कि बरात याद आयी, जिसमें बीस साल पहले वोह गया था। उस दावत में उसे जो तृप्ति मिली थी, वो उसके जीवन में एक याद रखने लायक बात थी, और आज भी उसकी याद ताज़ा थी।

बोला- वह भोज नही भूलता। तबसे फिर उस तरह का खाना और भर पेट नही मिला। लड्किवालों ने सबको भरपेट पूड़ीयां खिलायी थी, सबको!

छोटे-बडे सबने पूडीयां खायी और असली घी कि! चटनी, रीता, तीन तरह के सूखे साग, एक रसेदार तरकारी, दही, चटनी, मिठाई, अब क्या बताऊँ कि उस भोग में क्या स्वाद मिल, कोई रोक-टोक नहीं थी, जो चीज़ चाहो, मांगो, जितना चाहो खाओ। लोगों ने ऐसा खाया, ऐसा खाया, कि किसी से पानी न पीया गया। मगर परोसने वाले हैं कि पत्तल में गरम-गरम गोल-गोल सुवासित कचौद्दीयां डाल देते हैं। मन करते हैं कि नहीं चाहिऐ, पत्तल को हाथ से रोके हुए हैं, मगर वह हैं कि दिए जाते हैं और जब सबने मुँह धो लिया, तो पान एलैची भी मिली। मगर मुझे पान लेने की कहाँ सुध थी! खङा हुआ ना जाता था। झटपट अपने कम्बल पर जाकर लेट गया। ऐसा दिल दरियाव था वह ठाकुर!

माधव नें पदार्थों का मन ही मन मज़ा लेते हुए कहा- अब हमें कोई ऐसा भोजन नही खिलाता। ‘अब कोई क्या खिलायेगा। वह ज़माना दूसरा था। अब तो सबको किफायत सूझती है। शादी-ब्याह में मत खर्च करो। क्रिया-कर्म में मत खर्च करो। पूछों, गरीबों का माल बटोर-बटोर कर कहाँ रखोगे? बटोरने में तो कामं नही है। हाँ , खर्च में किफायती सूझती है। ‘

‘तुमने बीस-एक पूड़ीयां खायी होंगी?’

‘बीस से ज़्यादा खायी थी!’

‘मैं पचास खा जाता!’

‘पचास से कम मैंने भी ना खायी होगी। अच्छा पट्ठा था । तू तो मेरा आधा भी नही है ।’

आलू खाकर दोनों ने पानी पिया और वहीँ अलाव के सामने अपनी धोतियाँ ओढ़्कर पाँव पेट पर डाले सो रहे। जैसे दो बडे-बडे अजगर गेदुलियाँ मारे पडे हो।

और बुधिया अभी तक कराह रही थी।

सबेरे माधव ने कोठरी में जाकर देखा, तो उसकी स्त्री ठण्डी हो गयी थी। उसके मुँह पर मक्खियां भिनक रही थी। पथ्रायी हुई आँखें ऊपर टंगी हुई थी । साड़ी देह धुल से लथपथ हो रही थी थी। उसके पेट में बच्चा मर गया था।

माधव भागा हुआ घीसू के पास आया। फिर दोनों ज़ोर-ज़ोर से है-है करने और छाती पीटने लगे। पडोस्वालों ने यह रोना धोना सुना, तो दौड हुए आये और पुरानी मर्यादा के अनुसार इन अभागों को समझाने लगे।

मगर ज़्यादा रोने-पीटने का अवसर ना था। कफ़न और लकड़ी की फिक्र करनी थी। घर में तो पैसा इस तरह गायब था, जैसे चील के घोसले में मॉस!

बाप-बेटे रोते हुए गाव के ज़मिन्दार के पास गए। वह इन दोनों की सूरत से नफरत करते थे। कयी बार इन्हें अपने हाथों से पीट चुके थे। चोरी करने के लिए, वाडे पर काम पर न आने के लिए। पूछा- क्या है बे घिसुआ, रोता क्यों है? अब तो तू कहीँ दिखलायी भी नहीं देता! मालूम होता है, इस गाव में रहना नहीं चाहता।

घीसू ने ज़मीन पर सिर रखकर आंखों से आँसू भरे हुए कहा – सरकार! बड़ी विपत्ति में हूँ। माधव कि घर-वाली गुज़र गयी। रात-भर तड़पती रही सरकार! हम दोनों उसके सिरहाने बैठे रहे। दवा दारु जो कुछ हो सका, सब कुछ किया, पर वोह हमें दगा दे गयी। अब कोई एक रोटी देने वाला भी न रह मालिक! तबाह हो गए । घर उजाड़ गया। आपका घुलाम हूँ, अब आपके सिवा कौन उसकी मिटटी पार लगायेगा। हमारे हाथ में जो कुछ था, वोह सब तो दवा दारु में उठ गया…सरकार कि ही दया होगी तो उसकी मिटटी उठेगी। आपके सिवा किसके द्वार पर जाऊं!

ज़मीन्दार साहब दयालु थे। मगर घीसू पर दया करना काले कम्बल पर रंग चढाना था। जीं में तो आया, कह दे, चल, दूर हो यहाँ से। यों तोबुलाने से भी नही आता, आज जब गरज पढी तो आकर खुशामद कर रह है। हरामखोर कहीँ का, बदमाश! लेकिन यह क्रोध या दण्ड का अवसर न था। जीं में कूदते हुए दो रुपये निकालकर फ़ेंक दिए। मगर सांत्वना का एक शब्द भी मुँह से न निकला। उसकी तरफ ताका तक नहीं। जैसे सिर के बोझ उतारा हो। जब ज़मींदर साहब ने दो रुपये दिए, तो गाव के बनिए-महाजनों को इनकार का सहस कैसे होता? घीसू ज़मीन्दार का ढिंढोरा भी पीटना जानता था। किसी ने दो आने दिए, किसी ने चार आने। एक घंटे में घीसू और माधव बाज़ार से कफ़न लाने चले। इधर लोग बांस-वांस काटने लगे।

गाव की नर्म दिल स्त्रियां आ-आकर लाश देखती थी, और उसकी बेबसी पर दो बूँद आँसू गिराकर चली जाती थी।

बाज़ार में पहुंचकर, घीसू बोला – लकड़ी तो उसे जलाने भर कि मिल गयी है, क्यों माधव! माधव बोला – हाँ, लकड़ी तो बहुत है, अब कफ़न चाहिऐ।

‘तो चलो कोई हल्का-सा कफ़न ले लें।

‘हाँ, और क्या! लाश उठते उठते रात हो जायेगी। रात को कफ़न कौन देखता है!’

‘कैसा बुरा रिवाज है कि जिसे जीते-जीं तन धांकने को चीथ्डा भी न मिले, उसे मरने पर नया कफ़न चाहिऐ।’

‘कफ़न लाश के साथ जल ही तो जाता है।’

‘क्या रखा रहता है! यहीं पांच रुपये पहले मिलते, तो कुछ दवा-दारु कर लेते।

दोनों एक दुसरे के मॅन कि बात ताड़ रहे थे। बाज़ार में इधर-उधर घुमते रहे। कभी इस बजाज कि दुकान पर गए, कभी उस दुकान पर! तरह-तरह के कपडे, रेशमी और सूती देखे, मगर कुछ जंचा नहीं. यहाँ तक कि शाम हो गयी. तब दोनों न-जाने किस दयवी प्रेरणा से एक मधुशाला के सामने जा पहुंचे और जैसे पूर्व-निश्चित व्यवस्था से अन्दर चले गए. वहाँ ज़रा देर तक दोनों असमंजस में खडे रहे. फिर घीसू ने गड्डी के सामने जाकर कहा- साहूजी, एक बोतल हमें भी देना। उसके बाद कुछ चिखौना आया, तली हुई मछ्ली आयी, और बरामदे में बैठकर शांतिपूर्वक पीने लगे। कई कुज्जियां ताबड़्तोड़ पीने के बाद सुरूर में आ गए. घीसू बोला – कफ़न लगाने से क्या मिलता? आख़िर जल ही तो जाता. कुछ बहु के साथ तो न जाता. माधव आसमान कि तरफ देखकर बोला, मानो देवताओं को अपनी निश्पाप्ता का साक्षी बाना रह हो – दुनिया का दस्तूर है, नहीं लोग बाम्नों को हज़ारों रुपये क्यों दे देते हैं? कौन देखता है, परलोक में मिलता है या नहीं!

‘बडे आदमियों के पास धन है,फूंके. हमारे पास फूंकने को क्या है!’

‘लेकिन लोगों को जवाब क्या दोगे? लोग पूछेंगे नहीं, कफ़न कहाँ है?’

घीसू हसा – अबे, कह देंगे कि रुपये कंमर से खिसक गए। बहुत ढूंदा, मिले नहीं. लोगों को विश्वास नहीं आएगा, लेकिन फिर वही रुपये देंगे। माधव भी हंसा – इन अनपेक्षित सौभाग्य पर. बोला – बड़ी अच्छी थी बेचारी! मरी तो ख़ूब खिला पिला कर!

आधी बोतल से ज़्यादा उड़ गयी। घीसू ने दो सेर पूड़ियाँ मंगायी. चटनी, आचार, कलेजियां. शराबखाने के सामने ही दुकान थी. माधव लपककर दो पत्तलों में सारे सामान ले आया. पूरा डेड रुपया खर्च हो गया. सिर्फ थोड़े से पैसे बच रहे. दोनो इस वक़्त इस शान से बैठे पूड़ियाँ खा रहे थे जैसे जंगल में कोई शेर अपना शिकार उड़ रह हो. न जवाबदेही का खौफ था, न बदनामी का फिक्र. इन सब भावनाओं को उन्होने बहुत पहले ही जीत लिया था.

घीसू दार्शनिक भाव से बोला – हमारी आत्म प्रसन्न हो रही है तो क्या उसे पुन्न न होगा? माधव ने श्रध्दा से सिर झुकाकर तस्दीख कि – ज़रूर से ज़रूर होगा. भगवान्, तुम अंतर्यामी हो. उसे बय्कुंथ ले जान. हम दोनो हृदय से आशीर्वाद दे रहे हैं. आज जो भोजन मिल वोह कहीँ उम्र-भर न मिल था. एक क्षण के बाद मॅन में एक शंका जागी. बोला – क्यों दादा, हम लोग भी एक न एक दिन वहाँ जायेंगे ही? घीसू ने इस भोले-भाले सवाल का कुछ उत्तर न दिया. वोह परलोक कि बाते सोचकर इस आनंद में बाधा न डालना चाहता था।

‘जो वहाँ हम लोगों से पूछे कि तुमने हमें कफ़न क्यों नही दिया तो क्या कहेंगे?’

‘कहेंगे तुम्हारा सिर!’

‘पूछेगी तो ज़रूर!’

‘तू कैसे जानता है कि उसे कफ़न न मिलेगा? तू मुझेईसा गधा समझता है? साठ साल क्या दुनिया में घास खोदता रह हूँ? उसको कफ़न मिलेगा और बहुत अच्छा मिलेगा!’ माधव को विश्वास न आया। बोला – कौन देगा? रुपये तो तुमने चाट कर दिए। वह तो मुझसे पूछेगी। उसकी माँग में तो सिन्दूर मैंने डाला था।

घीसू गरम होकर बोला – मैं कहता हूँ, उसे कफ़न मिलेगा, तू मानता क्यों नहीं?

‘कौन देगा, बताते क्यों नहीं?’ ‘वही लोग देंगे, जिन्होंने अबकी दिया । हाँ, अबकी रुपये हमारे हाथ न आएंगे। ‘

ज्यों-ज्यों अँधेरा बढता था और सितारों की चमक तेज़ होती थी, मधुशाला, की रोनक भी बढती जाती थी। कोई गाता था, दींग मारता था, कोई अपने संगी के गले लिपट जाता था। कोई अपने दोस्त के मुँह में कुल्हड़ लगाए देता था। वहाँ के वातावरण में सुरूर था, हवा में नशा। कितने तो यहाँ आकर एक चुल्लू में मस्त हो जाते थे। शराब से ज़्यादा यहाँ की हवा उन पर नशा करती थी। जीवन की बाधाये यहाँ खीच लाती थी और कुछ देर के लिए यह भूल जाते थे कि वे जीते हैं कि मरते हैं। या न जीते हैं, न मरते हैं। और यह दोनो बाप बेटे अब भी मज़े ले-लेकर चुस्स्कियां ले रहे थे। सबकी निगाहें इनकी और जमी हुई थी। दोनों कितने भाग्य के बलि हैं! पूरी बोतल बीच में है।

भरपेट खाकर माधव ने बची हुई पूडियों का पत्तल उठाकर एक भिखारी को दे दिया, जो खडा इनकी और भूखी आंखों से देख रह था। और देने के गौरव, आनंद, और उल्लास का अपने जीवन में पहली बार अनुभव किया।

घीसू ने कहा – ले जा, ख़ूब खा और आर्शीवाद दे। बीवी कि कमायी है, वह तो मर गयी। मगर तेरा आर्शीवाद उसे ज़रूर पहुंचेगा। रोएँ-रोएँ से आर्शीवाद दो, बड़ी गाडी कमायी के पैसे हैं!

माधव ने फिर आसमान की तरफ देखकर कहा – वह बैकुंठ में जायेगी दादा, बैकुंठ की रानी बनेगी।

घीसू खड़ा हो गया और उल्लास की लहरों में तैरता हुआ बोला – हाँ बीटा, बैकुंठ में जायेगी। किसी को सताया नहीं, किसी को दबाया नहीं। मरते-मरते हमारी जिन्दगी की सबसे बड़ी लालसा पूरी कर गयी। वह न बैकुंठ जायेगी तो क्या मोटे-मोटे लोग जायेंगे, जो गरीबों को दोनों हाथों से लूटते हैं, और अपने पाप को धोने के लिए गंगा में नहाते हैं और मंदिरों में जल चडाते हैं?

श्रद्धालुता का यह रंग तुरंत ही बदल गया। अस्थिरता नशे की खासियत है। दु:ख और निराशा का दौरा हुआ। माधव बोला – मगर दादा, बेचारी ने जिन्दगी में बड़ा दु:ख भोगा। कितना दु:ख झेलकर मरी!

वह आंखों पर हाथ रखकर रोने लगा, चीखें मार-मारकर।

घीसू ने समझाया – क्यों रोता हैं बेटा, खुश हो कि वह माया-जाल से मुक्त हो गई, जंजाल से छूट गयी। बड़ी भाग्यवान थी, इतनी जल्द माया-मोह के बन्धन तोड़ दिए।

और दोनों खडे होकर गाने लगे – ”ठगिनी क्यों नैना झाम्कावे! ठगिनी …!”

पियाक्क्ड्डों की आँखें इनकी और लगी हुई थी और वे दोनो अपने दिल में मस्त गाये जाते थे। फिर दोनों नाचने लगे। उछले भी, कूदे भी। गिरे भी, मटके भी। भाव भी बनाए, अभिनय भी किये, और आख़िर नशे से मदमस्त होकर वहीँ गिर पडे।'''

In [8]:
# converting story into english using model
en_story = indic2en_model.translate_paragraph(story, 'hi', 'en')

100%|██████████| 230/230 [00:00<00:00, 9409.50it/s]


# Final Translated Story

In [9]:
en_story

'At the door of the hut both the father and the son lay buried, and inside, the young wife of the son, Budhia, was pining with pangs of labour. There was such a heart - wrenching sound coming out of his mouth that both of them held the collage together. It was a winter night, nature was in a state of silence, the whole village was in darkness. Ghisu said, I know it will not survive. I kept running all day long, come see. Madhav said angrily, If you have to die, why don \'t you die soon? What to see? You are toh very ajeeb! Such was the infidelity of the one with whom he or she had a good relationship for a whole year! "" "I don\'t see him shaking his hands and legs." There was a family of Chamars and a bad name in the whole village. If Ghisu worked one day, he would rest for three days. Madhav was so busy that if he worked for half an hour, he would drink for an hour. They were not getting wages for their work. If there was even a handful of grain in the house, he had sworn to work for